<a href="https://colab.research.google.com/github/Krishnan9074/EcommReact/blob/master/airbnb_DC_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly scikit-learn ipywidgets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor
from ipywidgets import interact, widgets
from IPython.display import display, HTML

In [ ]:
print("📁 Loading data...")
try:
    # You'll need to upload 'clean_airbnb_dc.csv' to Colab
    # Use: files.upload() or mount Google Drive
    df = pd.read_csv('clean_airbnb_dc.csv')
    print(f"✅ Loaded {len(df):,} listings successfully!")
except FileNotFoundError:
    print("⚠️ File 'clean_airbnb_dc.csv' not found.")
    print("Please upload the file using:")
    print("  from google.colab import files")
    print("  uploaded = files.upload()")
    raise

📁 Loading data...
✅ Loaded 4,842 listings successfully!


In [ ]:
print("\n🤖 Training predictive model...")
features = ['bedrooms', 'accommodates', 'dist_to_mall', 'number_of_reviews']
X = df[features]
y = df['price']
model = RandomForestRegressor(n_estimators=50, random_state=42)
model.fit(X, y)
print("✅ Model trained successfully!")



🤖 Training predictive model...
✅ Model trained successfully!


In [ ]:
print("\n" + "="*60)
print("🏠 THE CAPITAL INVESTOR - MARKET OVERVIEW")
print("="*60)
print(f"\n📊 Total Active Listings: {len(df):,}")
print(f"💵 Average Nightly Price: ${df['price'].mean():.2f}")
print(f"💵 Median Nightly Price: ${df['price'].median():.2f}")
print(f"📍 Average Distance to Mall: {df['dist_to_mall'].mean():.1f} miles")
print(f"🛏️  Average Bedrooms: {df['bedrooms'].mean():.1f}")
print(f"👥 Average Guest Capacity: {df['accommodates'].mean():.1f}")
print(f"\n⚠️  Note: Outliers will be removed for visualizations (IQR method)")



🏠 THE CAPITAL INVESTOR - MARKET OVERVIEW

📊 Total Active Listings: 4,842
💵 Average Nightly Price: $392.45
💵 Median Nightly Price: $135.00
📍 Average Distance to Mall: 2.6 miles
🛏️  Average Bedrooms: 1.5
👥 Average Guest Capacity: 3.6

⚠️  Note: Outliers will be removed for visualizations (IQR method)


In [ ]:
print("\n📊 Generating Market Insights Dashboard...\n")

# Visualization 1: Geospatial Price Heatmap
fig_map = px.scatter_mapbox(
    df,
    lat="latitude",
    lon="longitude",
    color="price",
    size="price",
    color_continuous_scale=px.colors.cyclical.IceFire,
    size_max=15,
    zoom=10,
    mapbox_style="carto-positron",
    hover_name="neighbourhood_cleansed",
    hover_data={'price': ':.2f', 'bedrooms': True, 'accommodates': True},
    title="📍 Washington D.C. Listings - Price Heatmap",
    height=600
)
fig_map.update_layout(
    font=dict(size=14),
    title_font_size=20
)
fig_map.show()

# Visualization 2: Price by Neighborhood (Top 15) - WITH OUTLIER REMOVAL
print("\n💰 Top 15 Most Expensive Neighborhoods (Outliers Removed):")

# Remove extreme outliers using IQR method
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_clean = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
print(f"   Removed {len(df) - len(df_clean):,} outlier listings (from ${lower_bound:.0f} to ${upper_bound:.0f})")

# Calculate neighborhood stats with cleaned data
nbhd_stats = df_clean.groupby('neighbourhood_cleansed').agg({
    'price': ['mean', 'count']
}).reset_index()
nbhd_stats.columns = ['neighbourhood_cleansed', 'avg_price', 'count']

# Only include neighborhoods with at least 5 listings for reliability
nbhd_stats = nbhd_stats[nbhd_stats['count'] >= 5]
nbhd_stats = nbhd_stats.sort_values(by='avg_price', ascending=False).head(15)

fig_bar = px.bar(
    nbhd_stats.sort_values(by='avg_price'),
    x='avg_price',
    y='neighbourhood_cleansed',
    orientation='h',
    title="💰 Average Price by Neighborhood (Top 15, Outliers Removed)",
    color='avg_price',
    color_continuous_scale='Bluered',
    labels={'avg_price': 'Average Price ($)', 'neighbourhood_cleansed': 'Neighborhood'},
    height=500,
    hover_data={'count': True}
)
fig_bar.update_layout(
    font=dict(size=12),
    title_font_size=18,
    showlegend=False
)
fig_bar.show()

# Visualization 3: Price Distribution (Cleaned Data)
fig_hist = px.histogram(
    df_clean,
    x='price',
    nbins=50,
    title="📊 Price Distribution (Outliers Removed)",
    labels={'price': 'Nightly Price ($)', 'count': 'Number of Listings'},
    color_discrete_sequence=['#636EFA']
)
fig_hist.update_layout(
    font=dict(size=14),
    title_font_size=18,
    showlegend=False
)
fig_hist.show()

# Visualization 4: Bedrooms vs Price (Cleaned Data)
fig_box = px.box(
    df_clean,
    x='bedrooms',
    y='price',
    title="🛏️ Price Distribution by Number of Bedrooms (Outliers Removed)",
    labels={'bedrooms': 'Number of Bedrooms', 'price': 'Price ($)'},
    color='bedrooms',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig_box.update_layout(
    font=dict(size=14),
    title_font_size=18,
    showlegend=False
)
fig_box.show()




📊 Generating Market Insights Dashboard...




💰 Top 15 Most Expensive Neighborhoods (Outliers Removed):
   Removed 262 outlier listings (from $-68 to $360)


In [ ]:
print("\n" + "="*60)
print("🤖 INTERACTIVE ROI CALCULATOR")
print("="*60)
print("\nUse the sliders below to get real-time predictions:\n")

def predict_roi(bedrooms, accommodates, dist_to_mall, number_of_reviews):
    """Interactive prediction function"""

    # Create prediction
    input_data = pd.DataFrame({
        'bedrooms': [bedrooms],
        'accommodates': [accommodates],
        'dist_to_mall': [dist_to_mall],
        'number_of_reviews': [number_of_reviews]
    })

    pred_price = model.predict(input_data)[0]
    occupancy = 0.65
    monthly_revenue = pred_price * 30 * occupancy
    annual_revenue = monthly_revenue * 12
    implied_value = annual_revenue * 15  # 15x annual revenue multiple

    # Display results with styling
    display(HTML(f"""
    <div style='background-color: #f0f2f6; padding: 20px; border-radius: 10px; margin: 10px 0;'>
        <h2 style='color: #1f77b4; margin-top: 0;'>💵 Financial Projections</h2>

        <div style='display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 15px; margin: 20px 0;'>
            <div style='background: white; padding: 15px; border-radius: 8px; border-left: 4px solid #ff7f0e;'>
                <h4 style='margin: 0; color: #666;'>Predicted Nightly Rate</h4>
                <h2 style='margin: 10px 0; color: #ff7f0e;'>${pred_price:.2f}</h2>
            </div>
            <div style='background: white; padding: 15px; border-radius: 8px; border-left: 4px solid #2ca02c;'>
                <h4 style='margin: 0; color: #666;'>Est. Monthly Revenue</h4>
                <h2 style='margin: 10px 0; color: #2ca02c;'>${monthly_revenue:,.2f}</h2>
            </div>
            <div style='background: white; padding: 15px; border-radius: 8px; border-left: 4px solid #d62728;'>
                <h4 style='margin: 0; color: #666;'>Implied Asset Value</h4>
                <h2 style='margin: 10px 0; color: #d62728;'>${implied_value:,.0f}</h2>
            </div>
        </div>

        <div style='background: #fff3cd; padding: 15px; border-radius: 8px; border-left: 4px solid #ffc107;'>
            <h4 style='margin: 0 0 10px 0; color: #856404;'>💡 AI Strategy Recommendation:</h4>
            <p style='margin: 0; color: #856404; line-height: 1.6;'>
    """))

    # AI Strategy Logic
    if dist_to_mall < 1.0:
        strategy = "This property is in a <strong>High-Traffic Tourist Zone</strong>. Prioritize luxury amenities (e.g., high-end coffee machine, premium linens) to justify the premium rate."
    elif dist_to_mall > 4.0:
        strategy = "This property is in a <strong>Residential/Commuter Zone</strong>. Compete on price and offer 'Work-from-Home' setups (fast WiFi, monitors) to attract long-term stays."
    else:
        strategy = "This is a <strong>Balanced Location</strong>. Ideal for families. Stock the property with family-friendly gear (cribs, games) to maximize occupancy."

    display(HTML(f"""
                {strategy}
            </p>
        </div>

        <div style='margin-top: 15px; padding: 10px; background: #e7f3ff; border-radius: 8px;'>
            <small style='color: #004085;'>
                <strong>Assumptions:</strong> 65% occupancy rate, 15x annual revenue valuation multiple
            </small>
        </div>
    </div>
    """))

# Create interactive widgets
interact(
    predict_roi,
    bedrooms=widgets.IntSlider(min=1, max=6, step=1, value=2, description='Bedrooms:'),
    accommodates=widgets.IntSlider(min=1, max=12, step=1, value=4, description='Guests:'),
    dist_to_mall=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.5, description='Dist to Mall:'),
    number_of_reviews=widgets.IntSlider(min=0, max=500, step=10, value=50, description='Reviews:')
)



🤖 INTERACTIVE ROI CALCULATOR

Use the sliders below to get real-time predictions:



interactive(children=(IntSlider(value=2, description='Bedrooms:', max=6, min=1), IntSlider(value=4, descriptio…

<function __main__.predict_roi(bedrooms, accommodates, dist_to_mall, number_of_reviews)>

In [ ]:
print("\n" + "="*60)
print("📈 ADDITIONAL MARKET ANALYTICS")
print("="*60)

# Correlation heatmap
print("\n📊 Feature Correlation Analysis:")
corr_features = ['price', 'bedrooms', 'accommodates', 'dist_to_mall', 'number_of_reviews']
correlation_matrix = df[corr_features].corr()

fig_heatmap = px.imshow(
    correlation_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title="🔥 Feature Correlation Heatmap",
    labels=dict(color="Correlation"),
    aspect='auto'
)
fig_heatmap.update_layout(
    font=dict(size=12),
    title_font_size=18
)
fig_heatmap.show()

print("\n✅ Analysis Complete!")
print("\n📌 Next Steps:")
print("  • Adjust sliders in the ROI Calculator above")
print("  • Explore different property configurations")
print("  • Use insights for investment decisions")
print("\n" + "="*60)


📈 ADDITIONAL MARKET ANALYTICS

📊 Feature Correlation Analysis:



✅ Analysis Complete!

📌 Next Steps:
  • Adjust sliders in the ROI Calculator above
  • Explore different property configurations
  • Use insights for investment decisions

